In [1]:
%pip install wildlife-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 919.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install wildlife-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 10.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install timm

Note: you may need to restart the kernel to use updated packages.


In [4]:
from itertools import chain
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

import timm
import itertools
from torch.optim import SGD
from wildlife_tools.train import ArcFaceLoss, BasicTrainer , TripletLoss

2024-08-06 07:24:13.585032: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 07:24:13.585140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 07:24:13.697897: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**IPanda50 Dataset**

In [5]:


# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()



DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=2aa43d58-768f-4d94-a07d-bad329d67675
To: /kaggle/data/IPanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:05<00:00, 165MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /kaggle/data/IPanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 76.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /kaggle/data/IPanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 154MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/204M [00:00<?, ?B/s]

...

Epoch 18: Average Loss = 0.1955854296684265


Epoch 18: 100%|█████████████████████████████████████████████████████| 54/54 [01:28<00:00,  1.63s/it]


Epoch 19: Average Loss = 0.19826319813728333


Epoch 19: 100%|█████████████████████████████████████████████████████| 54/54 [01:28<00:00,  1.63s/it]

Epoch 20: Average Loss = 0.19680996239185333


In [6]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_IPanda50_retrained_checkpoint.pth")


In [7]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


100%|███████████████████████████████████████████████████████████████| 53/53 [01:00<00:00,  1.14s/it]


In [8]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)


{'cosine': array([[-0.02517136,  0.46618938,  0.10621435, ...,  0.41428626,
         0.35380605,  0.04320086],
       [ 0.04157169,  0.5460777 , -0.08031382, ...,  0.16820785,
         0.6844107 , -0.14792083],
       [-0.0627168 ,  0.05465455,  0.13370544, ...,  0.03285281,
         0.26425716, -0.08095322],
       ...,
       [ 0.12668093,  0.14969292, -0.08159777, ...,  0.05781694,
         0.1480492 , -0.10006993],
       [-0.08160634,  0.00846808,  0.07374428, ..., -0.05845805,
         0.13918804,  0.00830896],
       [-0.02426066,  0.10131115,  0.6356449 , ...,  0.1971657 ,
         0.03030281, -0.02112505]], dtype=float32)}


In [9]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)


Predictions for 100 test Images:-
 ['10_hexing' '37_xinger' '18_miaomiao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '28_sa' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '36_xingda' '34_xiaoqiao' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '44_yayi' '28_sa' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '34_xiaoqiao' '35_xilan' '08_fulai' '00_aibang' '28_sa' '11_jiaoao'
 '46_yazhu' '44_yayi' '14_maodou' '30_shuqing' '36_xingda' '38_xingfan'
 '33_wuyi' '10_hexing' '10_hexing' '00_aibang' '49_yuanrun' '35_xilan'
 '04_chengdui' '10_hexing' '39_xinghui' '36_xingda' '10_hexing'
 '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui' '10_hexing'
 '17_meibang' '00_aibang' '28_sa' '14_maodou' '15_maosun' '32_susu'
 '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomiao' '48_yongbang'
 

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [10]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))


Accuracy on IPanda50 data: 92.00%


In [11]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.9640476190476189
Recall: 0.92
F1 Score: 0.9211111111111111


**MacaqueFaces Dataset**

In [12]:
# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')
# Load dataset metadata
metadata = datasets.MacaqueFaces('../data/MacaqueFaces')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET MacaqueFaces: DOWNLOADING STARTED.


MacaqueFaces.zip: 12.0MB [00:00, 14.9MB/s]                            
MacaqueFaces_ImageInfo.csv: 410kB [00:00, 827kB/s]                             


DATASET MacaqueFaces: EXTRACTING STARTED.
DATASET MacaqueFaces: FINISHED.



Epoch 0:   0%|                                                               | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  98%|████████████████████████████████████████████████████▉ | 49/50 [01:15<00:01,  1.52s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 50/50 [01:15<00:00,  1.52s/it]


Epoch 1: Average Loss = 0.1718163937330246


...

Epoch 18: Average Loss = 0.15710097551345825


Epoch 18: 100%|█████████████████████████████████████████████████████| 50/50 [01:13<00:00,  1.47s/it]


Epoch 19: Average Loss = 0.14713405072689056


Epoch 19: 100%|█████████████████████████████████████████████████████| 50/50 [01:13<00:00,  1.47s/it]

Epoch 20: Average Loss = 0.13160353899002075


In [13]:

import torch

trainer.save("retrained_chks", file_name="tripletloss_MacaqueFaces_retrained_checkpoint.pth")

In [14]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|███████████████████████████████████████████████████████████████| 49/49 [00:28<00:00,  1.69it/s]

{'cosine': array([[0.35306308, 0.9281481 , 0.9094564 , ..., 0.09214859, 0.08824382,
        0.10833621],
       [0.34774375, 0.95749867, 0.94479614, ..., 0.14353333, 0.15559652,
        0.18131451],
       [0.36218786, 0.9684678 , 0.9563611 , ..., 0.17002882, 0.18282145,
        0.19717412],
       ...,
       [0.3173271 , 0.96006584, 0.9554193 , ..., 0.13212933, 0.1307062 ,
        0.1565159 ],
       [0.33819792, 0.9518367 , 0.9450941 , ..., 0.12252291, 0.11629605,
        0.14149931],
       [0.36506605, 0.9744916 , 0.95762205, ..., 0.16555838, 0.17239049,
        0.18975937]], dtype=float32)}


In [15]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [16]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_P * 100))

Accuracy on MacaqueFaces data: 100.00%


In [17]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**LionData Dateset**

In [18]:
# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')
# Load dataset metadata
metadata = datasets.LionData('../data/LionData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET LionData: DOWNLOADING STARTED.


main.zip: 495MB [00:20, 24.3MB/s]


DATASET LionData: EXTRACTING STARTED.
DATASET LionData: FINISHED.



Epoch 0:   0%|                                                                | 0/6 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.52s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.64s/it]


Epoch 1: Average Loss = 0.3153253495693207


...

Epoch 18: Average Loss = 0.26607516407966614


Epoch 18: 100%|███████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.65s/it]


Epoch 19: Average Loss = 0.26645955443382263


Epoch 19: 100%|███████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.65s/it]

Epoch 20: Average Loss = 0.2657260000705719


In [19]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [20]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|█████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.45it/s]

{'cosine': array([[0.59850556, 0.45959282, 0.61357474, ..., 0.6695832 , 0.6892921 ,
        0.52469766],
       [0.78530467, 0.38379818, 0.7011812 , ..., 0.46585107, 0.79929864,
        0.5010545 ],
       [0.41573763, 0.37899375, 0.52384824, ..., 0.56756055, 0.6021615 ,
        0.6643369 ],
       ...,
       [0.43302828, 0.6523546 , 0.5352071 , ..., 0.72161615, 0.5247428 ,
        0.5202329 ],
       [0.39359093, 0.39323217, 0.507957  , ..., 0.57655716, 0.6045318 ,
        0.8149848 ],
       [0.4478227 , 0.48974007, 0.52272856, ..., 0.71754867, 0.60694253,
        0.71562374]], dtype=float32)}


In [21]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Sebastian' 'Saba' 'Sangiki' 'Tigisi' 'Naengop' 'Moswen' 'Enderoni'
 'Enderoni' 'Spring' 'Chaimu' 'Maskio' 'Manzy' 'Saimutie' 'Saba' 'Saitoti'
 'Blackie' 'Spring' 'Etana' 'Enderoni' 'Kinna' 'Senteu' 'Enkume' 'Joy'
 'Lemuanik' 'Naibor' 'Romeo' 'Kibibi' 'Naisiae' 'Sadala' 'Manzy' 'Moswen'
 'Lucinda' 'Siena' 'Senteu' 'Napi' 'Kibibi' 'Tikki' 'Kioni' 'Chaimu'
 'Nesapuka' 'Samir' 'Jamal' 'Leia' 'Sebastian' 'Yieyio' 'Charm' 'Nguro'
 'Moswen' 'Kibibi' 'Naini' 'Enderoni' 'Lucinda' 'Kinna' 'Nuru'
 'Sikio-Kali' 'Mashavu' 'Tigisi' 'Nguro' 'Lilly' 'Manzy' 'Opi'
 'Sikio-Kali' 'Ngare' 'Doto' 'Nariku-Inkgera' 'Lemuanik' 'Nakato' 'Ngare'
 'Chaimu' 'Kinna' 'Shemsa' 'Olonyori' 'Mickey' 'White-Eye' 'Naimina'
 'Nasieku' 'White-Eye' 'Nuru' 'Kioni' 'Nasieku' 'Lilly' 'Tigisi' 'Naisiae'
 'Moswen' 'Kioni' 'Naini' 'Rescue' 'Kinna' 'Empurra' 'Arria' 'Blackie'
 'Simaloi' 'Lucinda' 'Summer' 'Shambe' 'Enderoni' 'Maskio' 'Enkume'
 'Tigisi' 'Ngare']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [22]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on LionData data: {:.2f}%".format(accuracy_P * 100))

Accuracy on LionData data: 15.00%


In [23]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.5003333333333333
Recall: 0.15
F1 Score: 0.1419047619047619


**NyalaDataSet**

In [24]:
# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')
# Load dataset metadata
metadata = datasets.NyalaData('../data/NyalaData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET NyalaData: DOWNLOADING STARTED.


main.zip: 495MB [00:19, 25.0MB/s]


DATASET NyalaData: EXTRACTING STARTED.
DATASET NyalaData: FINISHED.



Epoch 0:   0%|                                                               | 0/16 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  94%|██████████████████████████████████████████████████▋   | 15/16 [00:54<00:03,  3.48s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 16/16 [00:54<00:00,  3.43s/it]


Epoch 1: Average Loss = 0.29118895530700684


...

Epoch 18: Average Loss = 0.2417650669813156


Epoch 18: 100%|█████████████████████████████████████████████████████| 16/16 [00:52<00:00,  3.26s/it]


Epoch 19: Average Loss = 0.22416135668754578


Epoch 19: 100%|█████████████████████████████████████████████████████| 16/16 [00:51<00:00,  3.24s/it]

Epoch 20: Average Loss = 0.22945758700370789


In [25]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [26]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|███████████████████████████████████████████████████████████████| 15/15 [00:47<00:00,  3.16s/it]

{'cosine': array([[0.79038334, 0.25119707, 0.13826722, ..., 0.591159  , 0.12097352,
        0.8033403 ],
       [0.3963278 , 0.7758279 , 0.48799422, ..., 0.31318957, 0.48478696,
        0.34889925],
       [0.73672795, 0.4618261 , 0.3236602 , ..., 0.9276824 , 0.1587424 ,
        0.64290994],
       ...,
       [0.28635502, 0.7668365 , 0.5678383 , ..., 0.2121097 , 0.6098042 ,
        0.28405485],
       [0.74933064, 0.30364254, 0.20889974, ..., 0.77692777, 0.12606966,
        0.8183049 ],
       [0.11544433, 0.4671737 , 0.79013646, ..., 0.09497271, 0.86449116,
        0.11041747]], dtype=float32)}


In [27]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['101' '10' '57' '149' '144' '179' '147' '58' '48' '71' '87' '44' '33'
 '144' '232' '32' '131' '111' '20' '20' '116' '19' '32' '51' '260' '30'
 '111' '11' '260' '46' '88' '114' '9' '131' '114' '123' '247' '108' '152'
 '178' '27' '40' '121' '46' '8' '122' '114' '62' '144' '149' '18' '43'
 '27' '108' '82' '252' '104' '85' '32' '82' '27' '48' '63' '51' '124'
 '181' '65' '242' '11' '144' '113' '90' '60' '90' '127' '166' '90' '146'
 '27' '50' '20' '149' '23' '85' '161' '20' '30' '62' '165' '7' '65' '174'
 '149' '217' '33' '225' '29' '85' '40' '8']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [28]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy_P * 100))

Accuracy on NyalaData data: 18.00%


In [29]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.6083333333333333
Recall: 0.18
F1 Score: 0.18985714285714286


**StripeSpotter Dataset**

In [30]:
# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')
# Load dataset metadata
metadata = datasets.StripeSpotter('../data/StripeSpotter')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET StripeSpotter: DOWNLOADING STARTED.


data-20110718.zip: 71.7MB [00:00, 126MB/s]                            
data-20110718.z02: 100%|██████████| 78.6M/78.6M [00:00<00:00, 102MB/s] 
data-20110718.z01: 100%|██████████| 78.6M/78.6M [00:00<00:00, 143MB/s]


DATASET StripeSpotter: EXTRACTING STARTED.
 copying: data/
 copying: data/.DS_Store
 copying: data/images/
 copying: data/images/img-0000001.jpg
 copying: data/images/img-0000002.jpg
 copying: data/images/img-0000003.jpg
 copying: data/images/img-0000004.jpg
 copying: data/images/img-0000005.jpg
 copying: data/images/img-0000006.jpg
 copying: data/images/img-0000007.jpg
 copying: data/images/img-0000008.jpg
 copying: data/images/img-0000009.jpg
 copying: data/images/img-0000010.jpg
 copying: data/images/img-0000011.jpg
 copying: data/images/img-0000012.jpg
 copying: data/images/img-0000013.jpg
 copying: data/images/img-0000014.jpg
 copying: data/images/img-0000015.jpg
 copying: data/images/img-0000016.jpg
 copying: data/images/img-0000017.jpg
 copying: data/images/img-0000018.jpg
 copying: data/images/img-0000019.jpg
 copying: data/images/img-0000020.jpg
 copying: data/images/img-0000021.jpg
 copying: data/images/img-0000022.jpg
 copying: data/images/img-0000023.jpg
 copying: data/imag

Epoch 0:   0%|                                                                | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  86%|████████████████████████████████████████████████        | 6/7 [00:23<00:03,  3.77s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 7/7 [00:24<00:00,  3.51s/it]


Epoch 1: Average Loss = 0.2627438008785248


...

Epoch 18: Average Loss = 0.17704124748706818


Epoch 18: 100%|███████████████████████████████████████████████████████| 7/7 [00:24<00:00,  3.44s/it]


Epoch 19: Average Loss = 0.1780577450990677


Epoch 19: 100%|███████████████████████████████████████████████████████| 7/7 [00:23<00:00,  3.38s/it]

Epoch 20: Average Loss = 0.17658083140850067


In [31]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [32]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|█████████████████████████████████████████████████████████████████| 6/6 [00:20<00:00,  3.41s/it]

{'cosine': array([[-0.02241088,  0.27230656,  0.1930488 , ...,  0.07687466,
         0.2652765 ,  0.44629616],
       [ 0.04800216,  0.48006326,  0.4798489 , ...,  0.6123388 ,
         0.2178252 ,  0.16183515],
       [ 0.47774723,  0.14714071,  0.09519885, ...,  0.04039922,
         0.15001519,  0.01657989],
       ...,
       [ 0.37502947, -0.02617005, -0.07750648, ..., -0.0170657 ,
         0.11693738, -0.02975595],
       [ 0.32702544, -0.05711059, -0.01171974, ...,  0.07205453,
         0.15749693, -0.15800299],
       [ 0.00678507,  0.10123745, -0.05477787, ...,  0.07853457,
         0.19642892,  0.0019774 ]], dtype=float32)}


In [33]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['09_200' 'S09_161' '08_085' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' 'S09_067' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '09_212' 'OP05_625' 'OP05_625' '02_149'
 '01_230' '09_200' 'OP05_625' '01_661' 'OP04_225' '01_661' '09_212'
 'S09_174' '01_679' '01_661' 'S09_067' 'S09_111' 'S09_112' '09_212'
 'OP05_625' '02_161' '09_185' '04_008' '08_100' '01_460' '01_679'
 'S09_111' '01_679' '01_036' 'S09_067']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [34]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy_P * 100))

Accuracy on StripeSpotter data: 98.00%


In [35]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.9840000000000001
Recall: 0.98
F1 Score: 0.9757777777777779


**CZoo Dataset**

In [36]:
# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')
# Load dataset metadata
metadata = datasets.CZoo('../data/CZoo')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET CZoo: DOWNLOADING STARTED.


master.zip: 634MB [00:20, 31.5MB/s]


DATASET CZoo: EXTRACTING STARTED.
DATASET CZoo: FINISHED.



Epoch 0:   0%|                                                               | 0/17 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  94%|██████████████████████████████████████████████████▊   | 16/17 [00:27<00:01,  1.68s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 17/17 [00:28<00:00,  1.69s/it]


Epoch 1: Average Loss = 0.14777177572250366


...

Epoch 18: Average Loss = 0.1369324028491974


Epoch 18: 100%|█████████████████████████████████████████████████████| 17/17 [00:26<00:00,  1.55s/it]


Epoch 19: Average Loss = 0.11583796888589859


Epoch 19: 100%|█████████████████████████████████████████████████████| 17/17 [00:26<00:00,  1.56s/it]

Epoch 20: Average Loss = 0.12804877758026123


In [37]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [38]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|███████████████████████████████████████████████████████████████| 16/16 [00:15<00:00,  1.02it/s]

{'cosine': array([[ 0.8502511 ,  0.8718872 ,  0.8557354 , ..., -0.09397024,
        -0.05230109,  0.00772383],
       [ 0.8678544 ,  0.90013874,  0.87604403, ..., -0.1021499 ,
        -0.06034018, -0.03326898],
       [ 0.86197037,  0.8926725 ,  0.8693619 , ..., -0.12851536,
        -0.04101291,  0.02454877],
       ...,
       [ 0.9066254 ,  0.9452016 ,  0.91690326, ..., -0.11964206,
        -0.00948693, -0.02488633],
       [ 0.9160557 ,  0.942768  ,  0.921376  , ..., -0.13018948,
        -0.01596886, -0.01630965],
       [ 0.90904164,  0.9490662 ,  0.93105555, ..., -0.13887672,
        -0.01834186, -0.00278286]], dtype=float32)}


In [39]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Alex' 'Alex' 'Alex' 'Jahaga' 'Jahaga' 'Alex' 'Alex' 'Jahaga' 'Alex'
 'Jahaga' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex'
 'Alex' 'Alex' 'Alex' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Gertrudia' 'Gertrudia' 'Gertrudia' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Annett' 'Annett' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Al

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [40]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy_P * 100))

Accuracy on CZoo data: 100.00%


In [41]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**CowDataset**

In [42]:
# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')
# Load dataset metadata
metadata = datasets.CowDataset('../data/CowDataset')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET CowDataset: DOWNLOADING STARTED.


31210192: 4.45GB [02:34, 28.8MB/s]                            


DATASET CowDataset: EXTRACTING STARTED.
DATASET CowDataset: FINISHED.



Epoch 0:   0%|                                                               | 0/12 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 12/12 [03:20<00:00, 14.72s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 12/12 [03:20<00:00, 16.72s/it]


Epoch 1: Average Loss = 0.2312423437833786


...

Epoch 18: Average Loss = 0.1191549301147461


Epoch 18: 100%|█████████████████████████████████████████████████████| 12/12 [03:44<00:00, 18.68s/it]


Epoch 19: Average Loss = 0.10949257761240005


Epoch 19: 100%|█████████████████████████████████████████████████████| 12/12 [03:59<00:00, 19.98s/it]

Epoch 20: Average Loss = 0.09842154383659363


In [43]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_LionData_retrained_checkpoint.pth")

In [44]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

100%|███████████████████████████████████████████████████████████████| 11/11 [03:58<00:00, 21.71s/it]

{'cosine': array([[ 0.01484122, -0.00933392,  0.20698497, ..., -0.01763014,
         0.11606835,  0.10376152],
       [-0.09437343, -0.04190393, -0.01942529, ..., -0.0908636 ,
         0.13024493,  0.14800182],
       [ 0.06803481,  0.09967951,  0.12267254, ...,  0.00362335,
         0.04222474,  0.03644698],
       ...,
       [ 0.03624496, -0.03919508,  0.20521393, ..., -0.03770024,
         0.15624198,  0.11048074],
       [-0.10742251,  0.13220885, -0.0403563 , ...,  0.04033719,
        -0.12071176, -0.14945063],
       [-0.0043379 , -0.01744456, -0.01402217, ...,  0.14583501,
         0.6626047 ,  0.7769567 ]], dtype=float32)}


In [45]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['2' '9' '2' '13' '1' '6' '4' '7' '1' '4' '10' '9' '13' '3' '6' '10' '6'
 '4' '2' '13' '6' '2' '2' '9' '8' '8' '4' '7' '2' '12' '6' '6' '2' '4' '5'
 '2' '7' '9' '4' '5' '9' '11' '1' '8' '8' '1' '9' '4' '3' '7' '12' '1' '2'
 '5' '5' '9' '4' '1' '10' '3' '11' '12' '5' '3' '6' '5' '5' '2' '7' '11'
 '8' '5' '3' '9' '8' '7' '2' '1' '11' '1' '6' '3' '12' '6' '3' '8' '10'
 '11' '7' '6' '1' '8' '6' '3' '11' '10' '10' '2' '3' '13']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [46]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on CowDataset data: {:.2f}%".format(accuracy_P * 100))

Accuracy on CowDataset data: 100.00%


In [47]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


# **ON New Unseen Datasets**

**DogFaceNet Dataset**

In [48]:
# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata = datasets.DogFaceNet('../data/DogFaceNet')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET DogFaceNet: DOWNLOADING STARTED.


DogFaceNet_Dataset_224_1.zip: 75.6MB [00:01, 66.8MB/s]                            


DATASET DogFaceNet: EXTRACTING STARTED.
DATASET DogFaceNet: FINISHED.



Epoch 0:   0%|                                                               | 0/66 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  98%|█████████████████████████████████████████████████████▏| 65/66 [01:44<00:01,  1.59s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 66/66 [01:44<00:00,  1.59s/it]


Epoch 1: Average Loss = 0.1704774647951126


...

Epoch 18: Average Loss = 0.1360481083393097


Epoch 18: 100%|█████████████████████████████████████████████████████| 66/66 [01:38<00:00,  1.50s/it]


Epoch 19: Average Loss = 0.1322341412305832


Epoch 19: 100%|█████████████████████████████████████████████████████| 66/66 [01:38<00:00,  1.50s/it]

Epoch 20: Average Loss = 0.1358930617570877


In [49]:
import torch

trainer.save("retrained_chks", file_name="tripletloss_DogFaceNet_retrained_checkpoint.pth")

In [50]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)


100%|███████████████████████████████████████████████████████████████| 65/65 [00:38<00:00,  1.68it/s]


In [51]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 0.04244501,  0.4237255 , -0.00557632, ...,  0.38746178,
         0.41110587, -0.04455552],
       [-0.08577716, -0.00877863,  0.04336447, ...,  0.18823624,
         0.13756146, -0.02631321],
       [-0.03869987, -0.23981205,  0.15198198, ...,  0.02533266,
         0.12911275,  0.20736074],
       ...,
       [ 0.26885834,  0.03710634,  0.20746723, ...,  0.21898195,
        -0.07853767,  0.3082709 ],
       [-0.05053294,  0.30906096, -0.14614153, ...,  0.22894101,
         0.37267512,  0.01802698],
       [ 0.00765296,  0.2877312 , -0.06552657, ...,  0.28065103,
         0.3680476 , -0.04660582]], dtype=float32)}


In [52]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['884' '182' '919' '1122' '428' '378' '1063' '585' '470' '405' '764' '17'
 '1025' '1339' '432' '673' '1075' '274' '296' '234' '1272' '991' '485'
 '425' '280' '407' '258' '1248' '446' '815' '892' '448' '240' '251' '448'
 '930' '321' '210' '345' '763' '622' '1324' '158' '127' '512' '1254'
 '1139' '208' '608' '424' '293' '415' '825' '147' '373' '192' '333' '333'
 '1314' '957' '329' '89' '189' '100' '122' '631' '491' '646' '26' '1133'
 '878' '673' '174' '447' '1169' '458' '473' '149' '373' '622' '540' '919'
 '589' '913' '782' '1357' '361' '482' '1324' '953' '183' '200' '299' '191'
 '397' '187' '270' '563' '1020' '1074']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [53]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on DogFaceNet data: {:.2f}%".format(accuracy_P * 100))

Accuracy on DogFaceNet data: 83.00%


In [54]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 0.83
F1 Score: 0.8333333333333335
